In [5]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-jrri9av5/twint_4c4f82540b4d4135aa5b60d3811de964
  Running command git clone -q https://github.com/twintproject/twint.git /tmp/pip-install-jrri9av5/twint_4c4f82540b4d4135aa5b60d3811de964
  Running command git checkout -q origin/master


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
import twint

In [9]:
c = twint.Config()

c.Search = ['#jamescharles']
c.Limit = 50000
c.Store_csv = True
c.Since = '2019-01-01'
c.Output = "jamescharles.csv"

twint.run.Search(c)

1537389204874768384 2022-06-16 10:58:44 +0000 <arianas_whisk> @daily_simper @burningRed_23 @BluTashiro We have to wait till you're legal 😔 #jamescharles #tonylopez #zoelaverne
1537385259490238467 2022-06-16 10:43:03 +0000 <thecelebpost> Jeffree Star says James Charles, Tati Westbrook 'ruined' YouTube beauty community    #JamesCharles #TatiWestbrook #TheCelebPost  https://t.co/esPreGuOw6
1536836170696560640 2022-06-14 22:21:10 +0000 <bluewolf_may> James Charles when he sees a kid @jamescharles #jamescharles #flop  https://t.co/2KL5SGyAWc
1536497434267947008 2022-06-13 23:55:09 +0000 <tfs_max_> Can somebody explain to me... How is James Charles not cancelled yet? 🤔 #jamescharles
1536082338546741248 2022-06-12 20:25:43 +0000 <BarryAllanMcGh1> #FACEBOOKCLAN #TwitterPeeps #Instagram Keep in mind; this is your retirement/Pensions/Investment Portfolios, which are at stake.  #ItIsReal #WakeUp #Beadvised #Zendaya #JamesCharles get too a podium right, now. #100millionpeopleanhour #topofgoogle #i